# Creating your first agent

[![Your first agent](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mozilla-ai/any-agent/blob/main/docs/cookbook/your_first_agent.ipynb) 

If you're looking to build your first agent using a few simple tools, this is a great place to start. In this cookbook example, we will create and run a simple agent that has access to a few web tools. This can be easily expanded to add more advanced tools and features. 🚀

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent'

import nest_asyncio

nest_asyncio.apply()

## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, TinyAgent, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the [TinyAgent](frameworks/tinyagent.md) framework.  

### Pick an LLM

Regardless of which agent framework you choose, each framework supports any-llm, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use Mistral AI's mistral:mistral-small-latest LLM.

### Pick which tools to use

 In this tutorial, we will provide the agent with access to web searches and web page visits. In later examples we will show more advanced tool usage like Model Context Protocol (MCP) tool use or inter-agent communication using Agent-2-Agent (A2A).

In [ ]:
import os
from getpass import getpass

for key in ("MISTRAL_API_KEY", "TAVILY_API_KEY"):
    if key not in os.environ:
        print(f"{key} not found in environment!")
        api_key = getpass(f"Please enter your {key}: ")
        os.environ[key] = api_key
        print(f"{key} set for this session!")
    else:
        print(f"{key} found in environment.")

We will use Tavily web search. For it, you will need a (free) [Tavily API key](https://www.tavily.com/). Alternatively, you can import and use `search_web` (Duck Duck Go Search).

In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import search_tavily

agent = AnyAgent.create(
    "tinyagent",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(model_id="mistral:mistral-small-latest", tools=[search_tavily]),
)

## Run the Agent

Now we've configured our agent, so it's time to run it! Let's give it a simple task: find 5 trending new TV shows that were released recently.


In [ ]:
agent_trace = agent.run(
    "What are 5 tv shows that are trending in 2025? Check a few sites, and provide the name of the show, the exact release date, the genre, and a brief description of the show."
)

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [ ]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.tokens.total_tokens:,}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")